In [19]:
import pandas as pd 
import matplotlib as plt
import numpy as np
import re
import nltk
'''nltk.download('stopwords')'''
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from nltk.util import pr
from nltk.corpus import stopwords
stemmer=nltk.SnowballStemmer("english")
stopwords=set(stopwords.words("english"))
df=pd.read_csv("train.csv")
df

,Unnamed: 0.1,Unnamed: 0,count,hate_speech_count,offensive_language_count,neither_count,class,tweet
0,0,0,3,0,0,3,2,(this text should be deleted) !!! RT @mayasolo...
1,1,1,3,0,3,0,1,(this text should be deleted) !!!!! RT @mleew1...
2,2,2,3,0,3,0,1,(this text should be deleted) !!!!!!! RT @UrKi...
3,3,3,3,0,2,1,1,(this text should be deleted) !!!!!!!!! RT @C_...
4,4,4,6,0,6,0,1,(this text should be deleted) !!!!!!!!!!!!! RT...
...,...,...,...,...,...,...,...,...
24778,24778,24778,3,0,2,1,1,(this text should be deleted) you's a muthaf**...
24779,24779,24779,3,0,1,2,2,(this text should be deleted) you've gone and ...
24780,24780,24780,3,0,3,0,1,(this text should be deleted) young buck wanna...
24781,24781,24781,6,0,6,0,1,(this text should be deleted) youu got wild bi...


In [20]:
df['labels']=df['class'].map({1:"hate speech detected",0:"offencive language detected",2:"no hate and offencive speech"})
df
df['labels']

0        no hate and offencive speech
1                hate speech detected
2                hate speech detected
3                hate speech detected
4                hate speech detected
                     ...             
24778            hate speech detected
24779    no hate and offencive speech
24780            hate speech detected
24781            hate speech detected
24782    no hate and offencive speech
Name: labels, Length: 24783, dtype: object

In [21]:
df['tweet']

0        (this text should be deleted) !!! RT @mayasolo...
1        (this text should be deleted) !!!!! RT @mleew1...
2        (this text should be deleted) !!!!!!! RT @UrKi...
3        (this text should be deleted) !!!!!!!!! RT @C_...
4        (this text should be deleted) !!!!!!!!!!!!! RT...
                               ...                        
24778    (this text should be deleted) you's a muthaf**...
24779    (this text should be deleted) you've gone and ...
24780    (this text should be deleted) young buck wanna...
24781    (this text should be deleted) youu got wild bi...
24782    (this text should be deleted) ~~Ruffled | Ntac...
Name: tweet, Length: 24783, dtype: object

In [22]:
df=df[['tweet','labels']]
df

,tweet,labels
0,(this text should be deleted) !!! RT @mayasolo...,no hate and offencive speech
1,(this text should be deleted) !!!!! RT @mleew1...,hate speech detected
2,(this text should be deleted) !!!!!!! RT @UrKi...,hate speech detected
3,(this text should be deleted) !!!!!!!!! RT @C_...,hate speech detected
4,(this text should be deleted) !!!!!!!!!!!!! RT...,hate speech detected
...,...,...
24778,(this text should be deleted) you's a muthaf**...,hate speech detected
24779,(this text should be deleted) you've gone and ...,no hate and offencive speech
24780,(this text should be deleted) young buck wanna...,hate speech detected
24781,(this text should be deleted) youu got wild bi...,hate speech detected


In [23]:
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+','', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split() if word not in stopwords]
    text = " ".join(text)
    return text
df["tweet"] = df["tweet"].apply(clean)
df

C:\Users\user\AppData\Local\Temp\ipykernel_12588\2304732816.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["tweet"] = df["tweet"].apply(clean)


,tweet,labels
0,text deleted rt mayasolovely woman shouldnt co...,no hate and offencive speech
1,text deleted rt boy dats coldtyga dwn bad cuff...,hate speech detected
2,text deleted rt urkindofbrand dawg rt ever fuc...,hate speech detected
3,text deleted rt cganderson vivabased look like...,hate speech detected
4,text deleted rt shenikaroberts shit hear might...,hate speech detected
...,...,...
24778,text deleted yous muthafin lie coreyemanuel ri...,hate speech detected
24779,text deleted youve gone broke wrong heart baby...,no hate and offencive speech
24780,text deleted young buck wanna eat dat nigguh l...,hate speech detected
24781,text deleted youu got wild bitches tellin lies,hate speech detected


In [24]:
df.dropna(axis=0, subset=['labels'], inplace=True) 

C:\Users\user\AppData\Local\Temp\ipykernel_12588\2321045353.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0, subset=['labels'], inplace=True)


In [25]:
# x=df[['tweet']]
# y=np.array(df[['labels']])
# cv=CountVectorizer()
# X=cv.fit_transform(x)
# #Y=cv.fit_transform(y)
# X_train,X_test,Y_train,Y_test=train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=0)
# Clf=DecisionTreeClassifier()
# Clf.fit(X_train,Y_train)


In [26]:
# testdata=("I will kill you")
# df=cv.transform([testdata]).toarray()
# print(Clf.predict(df))

In [40]:
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import BaggingClassifier 
from sklearn.svm import SVC
tweets = np.array(df["tweet"])
labels = np.array(df["labels"])
#labels2=np.array(df["class"])
# Transform tweet data into feature matrix
cv = CountVectorizer()
X = cv.fit_transform(tweets)

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, labels,train_size=0.8,test_size=0.2, random_state=42)
#X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X, labels,train_size=0.8,test_size=0.2, random_state=42)
# Initialize and train the classifier
clf =RandomForestClassifier(n_estimators=100,random_state=0)
#clf=BaggingClassifier(n_estimators=100,random_state=0)
#clf=SVC()
#clf= RandomForestClassifier()
clf.fit(X_train, Y_train)

RandomForestClassifier(random_state=0)

In [41]:
from sklearn.metrics import mean_absolute_error
pred=clf.predict((X_test))
print(pred)
#print(mean_absolute_error(Y_test,pred))

['hate speech detected' 'hate speech detected' 'hate speech detected' ...
 'hate speech detected' 'hate speech detected' 'hate speech detected']


In [42]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,pred)

0.8846076255799878

In [43]:
text1="fuck you"
text2="my friend"
text3='I am going to send you to hell'
text4='stop fooling around' 
text5=("I will kill you")
t=cv.transform(['nigga']).toarray()
print(clf.predict(t))


['hate speech detected']


In [44]:
import pickle as pk
with open("Hs_model.pkl", "wb") as file: # file is a variable for storing the newly created file, it can be anything.
    pk.dump(clf, file)
with open("Hs_cv.pkl", "wb") as file: # file is a variable for storing the newly created file, it can be anything.
    pk.dump(cv, file)    